In [16]:
import pandas as pd 
import numpy as np

In [334]:
#Set up fake data for individuals 
#age: the age of individual 
#sex: the sex of individual F:Female M:Male 
#Energydrink_consumption:Energy_drink consumption per week 

#Create list
IND = {"age":["59","54","35","73","49","30","59","54","35","73","49","30"],
      "sex":["F","F","F","F","F","F","M","M","M","M","M","M"],
      "Energydrink_consumption":["3","4","5","6","7","1","3","4","5","6","7","1"]}

#Create dataframe 
IND = pd.DataFrame(IND)

#Let age to become a numerical value
IND["age"] = IND["age"].apply(pd.to_numeric)

print(IND)

    age sex Energydrink_consumption
0    59   F                       3
1    54   F                       4
2    35   F                       5
3    73   F                       6
4    49   F                       7
5    30   F                       1
6    59   M                       3
7    54   M                       4
8    35   M                       5
9    73   M                       6
10   49   M                       7
11   30   M                       1


In [337]:
#Create aggreagted lists 

#Create emptylist based on constraints 
Under50 = []
Over50 = []
M = []
F = []

#For loop for each constraints -> binary outputs
for i in range(len(IND)):
    
    if IND.loc[i,"age"] < 50: 
            Under50.append(1)
            Over50.append(0)

    elif IND.loc[i,"age"] > 50: 
            Under50.append(0)
            Over50.append(1)

for i in range(len(IND)):

    if IND.loc[i,"sex"] == "M": 
            M.append(1)
            F.append(0)
            
    elif IND.loc[i,"sex"] == "F": 
            M.append(0)
            F.append(1)
            
        
#Combine all lists 
IND_Agg =[Under50,Over50,M,F]

print(IND_Agg)


[[0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]


In [527]:
IND_Agg = [[0,0,1,0,1],[1,1,0,1,0],[1,1,1,0,0],[0,0,0,1,1]]

In [697]:
#set up fake constraints 

#CONS = {"Under50":["8"],"Over50": ["4"],"M" : ["6"],"F" : ["6"]}
#CONS = pd.DataFrame(CONS) 
#for col in CONS: 
#    CONS[col] = CONS[col].apply(pd.to_numeric)

CONS = [[8,4,6,6],[2,8,4,6],[7,4,3,8]]

print(CONS)

[[8, 4, 6, 6], [2, 8, 4, 6], [7, 4, 3, 8]]


In [708]:
def IPF(X,Y,max_iteration):
    
    iteration = 0 
    
    weight = [1]*len(X[0])


    while iteration < max_iteration: 
        #Setting initial weight -> assuming weight are the same for 
        #each individual in an area 

        #K = 1....N(C)
        #N(C) the total number of constraints 
        #Target table 
        for k in range(len(Y)):

            #Ti_KM
            #in which the binary variable δjkm takes the value of one if cjk = m, 
            #and zero otherwise. In order to compare the modelled and observed distributions, a variety of goodness of fit measures could be employed, but typically the most useful will be the total absolute error statistic:
            X_new = pd.DataFrame(np.multiply(weight,X[k])).sum(axis=1)

            for i in range(len(X[k])):

                if X[k][i] == 1:

                    weight[i] = weight[i]*Y[k]/sum(X_new)

                else: 

                    weight[i] = weight[i]

        #return(weight)

        #print("Iteration " + str(k))

        Ti_km = pd.DataFrame(np.multiply(weight,X)).sum(axis=1)

    #   TAE = sum(abs(Y - Ti_km))
        TAE = sum(abs(Y - Ti_km))

        iteration += 1 
                    

        if (iteration == max_iteration):
        
            break
            
    return(abs(TAE),weight)





    
    #print(X_new)
    
    

In [675]:
a = 1
b = 1
while (a<10):
    print ('Iteration',a)
    a = a + 1
    b = b + 1
    c = b
    print(b)


    if (b == 4):
        break
print ('While loop terminated')

Iteration 1
2
Iteration 2
3
Iteration 3
4
While loop terminated


In [690]:
a = IPF(IND_Agg,CONS[0],10)
#predicted = pd.DataFrame(np.multiply(a[1],IND_Agg))
#sum(predicted - CONS[0])
#a[1]

#1.227998 1.227998 3.544004 1.544004 4.455996
a



(8.881784197001252e-16,
 [1.2279981273412341,
  1.2279981273412341,
  3.5440037453175317,
  1.5440037453175308,
  4.455996254682469])

In [559]:
CONS[0]

[8, 4, 6, 6]

In [560]:
pd.DataFrame(np.multiply(a[1],IND_Agg)).sum(axis=1)

0    8.1
1    3.9
2    6.0
3    6.0
dtype: float64

In [561]:
predicted = pd.DataFrame(np.multiply(a[1],IND_Agg)).sum(axis=1)


In [562]:
a[1]


[1.2000000000000002, 1.2000000000000002, 3.6000000000000005, 1.5, 4.5]

In [563]:
predicted


0    8.1
1    3.9
2    6.0
3    6.0
dtype: float64

In [564]:
CONS[0]



[8, 4, 6, 6]

In [709]:
#IPF2 - iteration all possible permutations -> find the best iteration with the smallest TAE

import itertools

def IPF2(X,Y,max_iteration = 1,max_iteration_IPF = 1):
    
    #Number of iteration for each permutation order 
    
    iteration = 0 
    
    smallest = 1000000
    
    smallest_iteration = 11
    
    #Generate all possible permutation for individual data 
    perm = list(itertools.permutations(X))
    
    #Generate all possible permutation for Constraint data 
    perm2 =list(itertools.permutations(Y))
    
    #While statement that loop through all iteration till it achieves max_iteration
    while iteration < max_iteration: 
        
        #Generate new individual data for each iteration 
        #For example, the first iteration has the following order: M, <50, F, >50 
        X_new = perm[iteration]

        #Generate new Constraints data for each iteration 
        #For example, the first iteration has the following order: M, <50, F, >50
        Y_new = perm2[iteration]
        
        #Note: The variables order of constraint and individual should be the same
        
        Result = IPF(X_new,Y_new,max_iteration_IPF)
        
        TAE = Result[0]
        
        Weight = Result[1]
        #print(TAE)
        
        print("Iteration: " + str(iteration) + " stoping criterion:" + " " + str(TAE))
        
        iteration += 1 
        
        if smallest > abs(TAE):
            
            smallest = abs(TAE)
            
            smallest_iteration = iteration 
            
            final_weight = Weight
            
    print("Converged! Smallest TAE: " + str(smallest) + " " + "at iteration: " + str(smallest_iteration - 1))

    return(final_weight)
    
            
#IPF2(IND_Agg,CONS,max_iteration = 24) 

In [716]:
a1 = IPF2(IND_Agg,CONS[2],max_iteration = 24,max_iteration_IPF =  20) 



Iteration: 0 stoping criterion: 0.0
Iteration: 1 stoping criterion: 0.0
Iteration: 2 stoping criterion: 5.213607323639735e-13
Iteration: 3 stoping criterion: 0.0
Iteration: 4 stoping criterion: 1.0125233984581428e-12
Iteration: 5 stoping criterion: 0.0
Iteration: 6 stoping criterion: 0.0
Iteration: 7 stoping criterion: 0.0
Iteration: 8 stoping criterion: 1.3988810110276972e-12
Iteration: 9 stoping criterion: 0.0
Iteration: 10 stoping criterion: 8.890665981198254e-13
Iteration: 11 stoping criterion: 0.0
Iteration: 12 stoping criterion: 0.0
Iteration: 13 stoping criterion: 1.0125233984581428e-12
Iteration: 14 stoping criterion: 0.0
Iteration: 15 stoping criterion: 1.297628671181883e-12
Iteration: 16 stoping criterion: 0.0
Iteration: 17 stoping criterion: 0.0
Iteration: 18 stoping criterion: 0.0
Iteration: 19 stoping criterion: 5.191402863147232e-13
Iteration: 20 stoping criterion: 0.0
Iteration: 21 stoping criterion: 3.1086244689504383e-13
Iteration: 22 stoping criterion: 0.0
Iteration: 

In [696]:
CONS[1]

[2, 8, 4, 8]

In [699]:
a1

#1.7250828 1.7250828 0.5498344 4.5498344 1.4501656



[1.725082782364625,
 1.725082782364625,
 0.54983443527075,
 4.54983443527075,
 1.45016556472925]

In [593]:
a2= IPF2(IND_Agg,CONS[2],max_iteration = 4) 


Iteration: 0 stoping criterion: 0.9916123019571295
Iteration: 1 stoping criterion: 0.9916123019571295
Iteration: 2 stoping criterion: 1.9920948616600795
Iteration: 3 stoping criterion: 2.262626262626262
Converged! Smallest TAE: 0.9916123019571295 at iteration: 0


In [594]:
p1  = pd.DataFrame(np.multiply(a1,IND_Agg)).sum(axis=1)
p2 = pd.DataFrame(np.multiply(a2,IND_Agg)).sum(axis=1)



In [597]:
CONS[2]

[7, 4, 3, 8]

In [595]:
p1

0    7.136585
1    4.130081
2    3.000000
3    8.266667
dtype: float64

In [596]:
p2


0    7.495806
1    3.504194
2    3.000000
3    8.000000
dtype: float64

In [606]:
a3 = [0.7250828, 0.7250828, 1.5498344, 2.5498344, 5.4501656]
p3 = pd.DataFrame(np.multiply(a3,IND_Agg)).sum(axis=1)


p3

0    7.0
1    4.0
2    3.0
3    8.0
dtype: float64

In [607]:
CONS[2]


[7, 4, 3, 8]

In [442]:
CONS[0] 


[8, 4, 6, 6]

In [443]:
#Return a complete weight matrix for each constraints and for each area based on the iteration with the smallest TAE
def IPF3(X,Y,Z = 1):
    
    area_weight = []
    
    for i in range(len(Y)):
        
        
        weight = IPF2(X,Y[i],max_iteration = Z)
        
        area_weight.append(weight)
        
    return(area_weight)
        


a =  IPF3(IND_Agg,CONS,24)
print(a)

Iteration: 0 stoping criterion: 0.666666666666667
Iteration: 1 stoping criterion: 0.6666666666666661
Iteration: 2 stoping criterion: 0.666666666666667
Iteration: 3 stoping criterion: 0.7999999999999998
Iteration: 4 stoping criterion: 0.5
Iteration: 5 stoping criterion: 0.7999999999999998
Iteration: 6 stoping criterion: 0.666666666666667
Iteration: 7 stoping criterion: 0.6666666666666661
Iteration: 8 stoping criterion: 1.6321839080459775
Iteration: 9 stoping criterion: 3.792207792207792
Iteration: 10 stoping criterion: 1.0933333333333337
Iteration: 11 stoping criterion: 3.792207792207792
Iteration: 12 stoping criterion: 2.533333333333333
Iteration: 13 stoping criterion: 1.6363636363636367
Iteration: 14 stoping criterion: 2.533333333333333
Iteration: 15 stoping criterion: 2.666666666666667
Iteration: 16 stoping criterion: 3.0
Iteration: 17 stoping criterion: 3.0
Iteration: 18 stoping criterion: 2.4
Iteration: 19 stoping criterion: 2.0
Iteration: 20 stoping criterion: 2.4
Iteration: 21 st